In [9]:
%load_ext autoreload
%autoreload 2
import os

os.environ["PREFECT__FLOWS__CHECKPOINTING"] = "True"

import fsspec
import intake
import pandas as pd
import xarray as xr
import xesmf as xe
import zarr
from prefect import Flow, Parameter, task
import re
import numpy as np
from xarray_schema import DataArraySchema, DatasetSchema

from cmip6_downscaling.workflows.utils import rechunk_zarr_array, calc_auspicious_chunks_dict, delete_chunks_encoding, regrid_dataset, load_paths
import cmip6_downscaling
import numpy as np
from xarray_schema import DataArraySchema, DatasetSchema
connection_string = "DefaultEndpointsProtocol=https;AccountName=cmip6downscaling;AccountKey=Z86Ca7uttNw4smgPikZFgsB1Lgf8GFpL+lMiViWRaovBSsOWK5JFXAvlk5rO18R/61wqSfUDolvd8olXLt0law==;EndpointSuffix=core.windows.net"

os.environ['AZURE_STORAGE_CONNECTION_STRING'] = connection_string
from cmip6_downscaling.data.observations import open_era5
from cmip6_downscaling.data.cmip import load_cmip, gcm_munge, convert_to_360
from cmip6_downscaling.methods.bcsd import preprocess_bcsd, prep_bcsd_inputs, fit_and_predict, postprocess_bcsd, make_flow_paths
from dask.distributed import Client, LocalCluster
import warnings
import rechunker
warnings.filterwarnings("ignore")
import pytest
from cmip6_downscaling import analysis
from cmip6_downscaling.analysis import load
from cmip6_downscaling.analysis.analysis import qaqc_checks
from cmip6_downscaling.analysis.qaqc import make_qaqc_ds
import matplotlib.pyplot as plt

In [3]:
cluster = LocalCluster()
client = Client(cluster)
cluster.scale(4)

In [4]:
# coarse_obs_path = f"az://cmip6/intermediates/{obs_id}_{gcm[0]}_{train_period_start}_{train_period_end}_{variable}.zarr"
# spatial_anomalies_path =  f"az://cmip6/intermediates/anomalies_{obs_id}_{gcm[0]}_{train_period_start}_{train_period_end}_{variable}.zarr"
# bias_corrected_path = f"az://cmip6/intermediates/bc_{obs_id}_{gcm[0]}_{train_period_start}_{train_period_end}_{variable}.zarr"

In [5]:
# bias_corrected_path = f"az://cmip6/intermediates/bc_{scenario}_{gcm[0]}_{train_period_start}_{train_period_end}_{variable}.zarr"

In [6]:
variable = 'tasmax'
gcm = "MIROC6"
obs_id = 'ERA5'
train_period_start='1990'
train_period_end='1990'
variable='tasmax'
out_bucket='cmip6'
predict_period_start='2079'
predict_period_end='2079'
scenario='ssp370'
downscale_method='bcsd'

In [7]:
ds = load.load_downscaled_ds(variable=variable, 
                                      gcm=gcm, 
                                      obs_id=obs_id,
                                      scenario=scenario,
                                      predict_period_start=predict_period_start,
                                      predict_period_end=predict_period_end,
                                      downscale_method=downscale_method)

In [6]:
# ds.to_zarr('test.zarr')

In [ ]:
# run_id = f"{downscale_method}_{scenario}_{gcm}_{predict_period_start}_{predict_period_end}_{variable}"

In [9]:
# ds -= 273.15
# qaqc_ds = make_qaqc_ds(ds)

In [44]:
annual_qaqc_ts, qaqc_maps = qaqc_checks(ds.isel(time=slice(0,2)))

okay


In [10]:
# # maps = qaqc_ds.sum(dim='time')
# fig, axarr = plt.subplots(ncols=len(maps), nrows=len(maps['qaqc_check']))
# for col, variable in enumerate(maps.data_vars):
#     for row, qaqc_check in enumerate(maps['qaqc_check']):
#         maps[variable].sel(qaqc_check=qaqc_check).plot(ax=axarr[row, col])
    

# Summary file creation - to be done using funnel/caching

In [5]:
test_tuple_one = (3,)
test_tuple_two = (3,4)

In [8]:
method = np.max

NameError: name 'np' is not defined

In [ ]:
summarizing_method = {'tmax': (np.max,
                      'tmin': np.min,
                     'wind': np.max,}
time_aggregations = ['month', 'year']
# @task(serializer=funnel, cache='summaries/same_string_as_daily_but_{time_aggregation})
# def create_time_summary(ds, time_aggregation):
# out_ds = xr.Dataset()
# for variable, method in summarizing_method:
#     out_ds[variable] = method(ds.groupby(f'time.{time_aggregation}'))
# return out_ds

# QAQC Routines

In [ ]:

returns dataset of same shape but with count of instances of bad data
'''
def timeseries_summary()
# timeseries (return table with sum through time for each three metrics) that checks for
* nans
* temp > 150, below -150
* precip > 100 ?? (placeholder since we don't have precip)

# Metrics

In [ ]:
# historical_ds = load_obs(start_year, end_year)
# def historical_skill_timeseries()

# def historical_skill_maps():
# xskillscore calculated along time dimension at different aggregations (

# def compare_with_obs_historical(ds, historical_ds):
#xskillscore

# Check for nans map
# plot count of each map